<h1>Applied Data Science Capstone</h1>
<hr>

## Data Acquisition and Preprocessing

In [1]:
import numpy as np
import pandas as pd

import json
!pip install geopy
from geopy.geocoders import Nominatim

import requests 
from pandas.io.json import json_normalize 

import matplotlib as plt
import matplotlib.cm as cm
import matplotlib.colors as colors

from bs4 import BeautifulSoup
from sklearn.cluster import KMeans

! pip install folium==0.5.0
import folium 

     |████████████████████████████████| 79 kB 6.6 MB/s  eta 0:00:01
  Created wheel for folium: filename=folium-0.5.0-py3-none-any.whl size=76240 sha256=a347c5322af309b0009e04e20da24e20b6b214d3b1d5cc6c15695aa970eef6fb
  Stored in directory: /tmp/wsuser/.cache/pip/wheels/b2/2f/2c/109e446b990d663ea5ce9b078b5e7c1a9c45cca91f377080f8
Successfully built folium


In [2]:
# Get the 18 districts of HKSAR from Wikipedia by web scraping
url = "https://en.wikipedia.org/wiki/Districts_of_Hong_Kong"
source = requests.get(url).text
BS = BeautifulSoup(source)
table = BS.find('div', class_='mw-parser-output')
table = table.table.tbody.p
district = []
for i in table.find_all('a'):
    try:
        district_split = i.get('title').split(",")
        district.append(district_split[0])
    except:
        district.append(i.get('title'))
print(district)

['Islands District', 'Kwai Tsing District', 'North District', 'Sai Kung District', 'Sha Tin District', 'Tai Po District', 'Tsuen Wan District', 'Tuen Mun District', 'Yuen Long District', 'Kowloon City District', 'Kwun Tong District', 'Sham Shui Po District', 'Wong Tai Sin District', 'Yau Tsim Mong District', 'Central and Western District', 'Eastern District', 'Southern District', 'Wan Chai District']


In [3]:
# Create dataframe
columns = ['District', 'Latitude', 'Longitude']
lat = np.zeros(len(district), dtype=int)
long = np.zeros(len(district), dtype=int)
hk = pd.DataFrame({columns[0]: district,
                   columns[1]: lat,
                   columns[2]: long})

In [4]:
# Get the coordinates of the districts by using geocoder
for district, a in zip(hk['District'], hk.index):
    address = '{}, Hong Kong'.format(district)
    geolocator = Nominatim(user_agent="my application")
    location = geolocator.geocode(address)
    latitude = round(location.latitude,4)
    longitude = round(location.longitude,4)
    hk.loc[a,'Latitude'] =  latitude
    hk.loc[a,'Longitude'] = longitude
hk.head()

,District,Latitude,Longitude
0,Islands District,22.2301,113.9868
1,Kwai Tsing District,22.3410,114.1043
2,North District,22.5169,114.2136
3,Sai Kung District,22.3070,114.3713
4,Sha Tin District,22.3916,114.2081


In [5]:
# Get the coordinate of HKSAR
address = 'Hong Kong'
geolocator = Nominatim(user_agent="my application")
location = geolocator.geocode(address)
latitude = round(location.latitude,4)
longitude = round(location.longitude,4)
print(latitude)
print(longitude)

22.2793
114.1628


In [6]:
# Inspect the coordinates of the 18 districts on the map
map_HK = folium.Map(location=[latitude+0.05, longitude], zoom_start=11)

for lat, lng, label in zip(hk['Latitude'], hk['Longitude'], hk['District']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_HK)  

map_HK

In [7]:
# Drop the unwanted districts 
hk.drop([0,2,3,5,6,7,8], inplace=True)
hk = hk.reset_index(drop=True)
hk

,District,Latitude,Longitude
0,Kwai Tsing District,22.3410,114.1043
1,Sha Tin District,22.3916,114.2081
2,Kowloon City District,22.3218,114.1886
3,Kwun Tong District,22.3086,114.2277
4,Sham Shui Po District,22.3313,114.1593
5,Wong Tai Sin District,22.3443,114.2021
6,Yau Tsim Mong District,22.3074,114.1655
7,Central and Western District,22.2748,114.1487
8,Eastern District,22.2731,114.2336
9,Southern District,22.2193,114.2252


In [8]:
# Inspect the selected districts on the map
map_HK = folium.Map(location=[latitude+0.025, longitude], zoom_start=12)

for lat, lng, label in zip(hk['Latitude'], hk['Longitude'], hk['District']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_HK)  

map_HK

<hr>

## Exploratory Data Analysis

In [9]:
CLIENT_ID = 'AJD2BORKJ3BYMYD3M3U0250PIL2P2LVW5HZEBKLHUYWBX24S' 
CLIENT_SECRET = 'L3EKD1YOG1XUOCH5LXKIRSICYZY4NLWGFLV5ZQDPURUYA2LI' 
VERSION = '20210101' 
LIMIT = 100 

In [10]:
def getNearbyVenues(names, latitudes, longitudes, radius=1000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['District', 
                  'District Latitude', 
                  'District Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [11]:
hk_venues = getNearbyVenues(names=hk['District'],
                                   latitudes=hk['Latitude'],
                                   longitudes=hk['Longitude'])

Kwai Tsing District
Sha Tin District
Kowloon City District
Kwun Tong District
Sham Shui Po District
Wong Tai Sin District
Yau Tsim Mong District
Central and Western District
Eastern District
Southern District
Wan Chai District


In [12]:
print(hk_venues.shape)
hk_venues.head()

(712, 7)


,District,District Latitude,District Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Kwai Tsing District,22.341,114.1043,Cheung Tsing Tunnel (長青隧道),22.345194,114.103667,Tunnel
1,Kwai Tsing District,22.341,114.1043,Café de Coral (大家樂),22.343849,114.111346,Fast Food Restaurant
2,Kwai Tsing District,22.341,114.1043,Fairwood 大快活,22.344454,114.108044,Restaurant
3,Kwai Tsing District,22.341,114.1043,Bangkok BBQ,22.344964,114.107789,Thai Restaurant
4,Kwai Tsing District,22.341,114.1043,Wellcome 惠康,22.344068,114.109194,Grocery Store


In [13]:
hk_venues.groupby('District').count().sort_values(['Venue'], ascending=False, inplace=False)

,District Latitude,District Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
District,,,,,,
Central and Western District,100,100,100,100,100,100
Wan Chai District,100,100,100,100,100,100
Yau Tsim Mong District,100,100,100,100,100,100
Kowloon City District,86,86,86,86,86,86
Sham Shui Po District,86,86,86,86,86,86
Kwun Tong District,82,82,82,82,82,82
Eastern District,61,61,61,61,61,61
Wong Tai Sin District,52,52,52,52,52,52
Sha Tin District,28,28,28,28,28,28


In [14]:
print('There are {} uniques categories.'.format(len(hk_venues['Venue Category'].unique())))

There are 158 uniques categories.


In [15]:
# one hot encoding
hk_onehot = pd.get_dummies(hk_venues[['Venue Category']], prefix="", prefix_sep="")

# add District column back to dataframe
hk_onehot['District'] = hk_venues['District'] 

# move District column to the first column
fixed_columns = [hk_onehot.columns[-1]] + list(hk_onehot.columns[:-1])
hk_onehot = hk_onehot[fixed_columns]

hk_onehot.head()

,District,American Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Asian Restaurant,Astrologer,Athletics & Sports,Australian Restaurant,BBQ Joint,...,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Watch Shop,Waterfall,Waterfront,Whisky Bar,Wine Bar,Xinjiang Restaurant,Yoga Studio,Zhejiang Restaurant
0,Kwai Tsing District,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Kwai Tsing District,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Kwai Tsing District,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Kwai Tsing District,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Kwai Tsing District,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [16]:
hk_onehot.shape

(712, 159)

In [17]:
hk_grouped = hk_onehot.groupby('District').mean().reset_index()
hk_grouped

,District,American Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Asian Restaurant,Astrologer,Athletics & Sports,Australian Restaurant,BBQ Joint,...,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Watch Shop,Waterfall,Waterfront,Whisky Bar,Wine Bar,Xinjiang Restaurant,Yoga Studio,Zhejiang Restaurant
0,Central and Western District,0.000000,0.00,0.020000,0.02,0.010000,0.000000,0.000000,0.01,0.010000,...,0.020000,0.020000,0.01,0.01,0.000000,0.00,0.02,0.000000,0.02,0.00
1,Eastern District,0.000000,0.00,0.000000,0.00,0.016393,0.000000,0.000000,0.00,0.016393,...,0.000000,0.000000,0.00,0.00,0.016393,0.00,0.00,0.000000,0.00,0.00
2,Kowloon City District,0.000000,0.00,0.011628,0.00,0.023256,0.000000,0.000000,0.00,0.000000,...,0.000000,0.011628,0.00,0.00,0.000000,0.00,0.00,0.000000,0.00,0.00
3,Kwai Tsing District,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,...,0.000000,0.000000,0.00,0.00,0.000000,0.00,0.00,0.000000,0.00,0.00
4,Kwun Tong District,0.000000,0.00,0.012195,0.00,0.000000,0.000000,0.000000,0.00,0.012195,...,0.000000,0.012195,0.00,0.00,0.000000,0.00,0.00,0.000000,0.00,0.00
5,Sha Tin District,0.035714,0.00,0.000000,0.00,0.000000,0.000000,0.071429,0.00,0.000000,...,0.000000,0.000000,0.00,0.00,0.000000,0.00,0.00,0.000000,0.00,0.00
6,Sham Shui Po District,0.000000,0.00,0.011628,0.00,0.000000,0.000000,0.000000,0.00,0.000000,...,0.000000,0.011628,0.00,0.00,0.000000,0.00,0.00,0.011628,0.00,0.00
7,Southern District,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,...,0.000000,0.000000,0.00,0.00,0.000000,0.00,0.00,0.000000,0.00,0.00
8,Wan Chai District,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,...,0.010000,0.010000,0.00,0.00,0.000000,0.01,0.00,0.000000,0.01,0.01
9,Wong Tai Sin District,0.000000,0.00,0.019231,0.00,0.019231,0.019231,0.038462,0.00,0.019231,...,0.019231,0.019231,0.00,0.00,0.000000,0.00,0.00,0.000000,0.00,0.00


In [18]:
hk_grouped.shape

(11, 159)

In [19]:
num_top_venues = 5

for hood in hk_grouped['District']:
    print("----"+hood+"----")
    temp = hk_grouped[hk_grouped['District'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Central and Western District----
                 venue  freq
0          Coffee Shop  0.09
1  Japanese Restaurant  0.05
2   Italian Restaurant  0.04
3    French Restaurant  0.03
4                  Bar  0.03


----Eastern District----
                  venue  freq
0          Noodle House  0.10
1  Fast Food Restaurant  0.07
2     Convenience Store  0.07
3        Cha Chaan Teng  0.05
4  Cantonese Restaurant  0.05


----Kowloon City District----
                  venue  freq
0       Thai Restaurant  0.13
1    Chinese Restaurant  0.12
2          Dessert Shop  0.10
3  Fast Food Restaurant  0.08
4                  Café  0.07


----Kwai Tsing District----
                  venue  freq
0                 Hotel  0.19
1     Convenience Store  0.12
2                Tunnel  0.12
3  Fast Food Restaurant  0.06
4         Grocery Store  0.06


----Kwun Tong District----
                  venue  freq
0    Chinese Restaurant  0.10
1                  Café  0.06
2  Fast Food Restaurant  0.06
3          

In [20]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [21]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['District']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
hk_venues_sorted = pd.DataFrame(columns=columns)
hk_venues_sorted['District'] = hk_grouped['District']

for ind in np.arange(hk_grouped.shape[0]):
    hk_venues_sorted.iloc[ind, 1:] = return_most_common_venues(hk_grouped.iloc[ind, :], num_top_venues)

hk_venues_sorted.head()

,District,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Central and Western District,Coffee Shop,Japanese Restaurant,Italian Restaurant,Café,Scenic Lookout,Indian Restaurant,French Restaurant,Bar,Bakery,Trail
1,Eastern District,Noodle House,Fast Food Restaurant,Convenience Store,Cantonese Restaurant,Cha Chaan Teng,Bakery,Tram Station,Snack Place,Shopping Mall,Farmers Market
2,Kowloon City District,Thai Restaurant,Chinese Restaurant,Dessert Shop,Fast Food Restaurant,Café,Coffee Shop,Cha Chaan Teng,Bakery,Cantonese Restaurant,Halal Restaurant
3,Kwai Tsing District,Hotel,Tunnel,Convenience Store,Thai Restaurant,Cantonese Restaurant,Chinese Restaurant,Mountain,Fast Food Restaurant,Grocery Store,Shopping Mall
4,Kwun Tong District,Chinese Restaurant,Café,Fast Food Restaurant,Coffee Shop,Cha Chaan Teng,Shopping Mall,Pizza Place,Japanese Restaurant,Cantonese Restaurant,Malay Restaurant


In [22]:
# set number of clusters
kclusters = 5

hk_grouped_clustering = hk_grouped.drop('District', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(hk_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([4, 0, 0, 2, 0, 3, 0, 1, 4, 0], dtype=int32)

In [23]:
# add clustering labels
hk_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

hk_merged = hk

# merge manhattan_grouped with manhattan_data to add latitude/longitude for each District
hk_merged = hk_merged.join(hk_venues_sorted.set_index('District'), on='District')

hk_merged.head() # check the last columns!

,District,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Kwai Tsing District,22.3410,114.1043,2,Hotel,Tunnel,Convenience Store,Thai Restaurant,Cantonese Restaurant,Chinese Restaurant,Mountain,Fast Food Restaurant,Grocery Store,Shopping Mall
1,Sha Tin District,22.3916,114.2081,3,Chinese Restaurant,Athletics & Sports,Convenience Store,Shopping Mall,Fast Food Restaurant,Pharmacy,Bus Stop,American Restaurant,Buffet,Donburi Restaurant
2,Kowloon City District,22.3218,114.1886,0,Thai Restaurant,Chinese Restaurant,Dessert Shop,Fast Food Restaurant,Café,Coffee Shop,Cha Chaan Teng,Bakery,Cantonese Restaurant,Halal Restaurant
3,Kwun Tong District,22.3086,114.2277,0,Chinese Restaurant,Café,Fast Food Restaurant,Coffee Shop,Cha Chaan Teng,Shopping Mall,Pizza Place,Japanese Restaurant,Cantonese Restaurant,Malay Restaurant
4,Sham Shui Po District,22.3313,114.1593,0,Noodle House,Chinese Restaurant,Shopping Mall,Fast Food Restaurant,Hong Kong Restaurant,Cha Chaan Teng,Dessert Shop,Dim Sum Restaurant,Snack Place,History Museum


In [24]:
# create map
map_clusters = folium.Map(location=[latitude+0.025, longitude], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(hk_merged['Latitude'], hk_merged['Longitude'], hk_merged['District'], hk_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster],
        fill=True,
        fill_color=rainbow[cluster],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [25]:
hk_merged.loc[hk_merged['Cluster Labels'] == 0, hk_merged.columns[[0] + list(range(3, hk_merged.shape[1]))]]

,District,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Kowloon City District,0,Thai Restaurant,Chinese Restaurant,Dessert Shop,Fast Food Restaurant,Café,Coffee Shop,Cha Chaan Teng,Bakery,Cantonese Restaurant,Halal Restaurant
3,Kwun Tong District,0,Chinese Restaurant,Café,Fast Food Restaurant,Coffee Shop,Cha Chaan Teng,Shopping Mall,Pizza Place,Japanese Restaurant,Cantonese Restaurant,Malay Restaurant
4,Sham Shui Po District,0,Noodle House,Chinese Restaurant,Shopping Mall,Fast Food Restaurant,Hong Kong Restaurant,Cha Chaan Teng,Dessert Shop,Dim Sum Restaurant,Snack Place,History Museum
5,Wong Tai Sin District,0,Fast Food Restaurant,Chinese Restaurant,Shopping Mall,Cha Chaan Teng,Athletics & Sports,Coffee Shop,Convenience Store,Garden,Pizza Place,Park
8,Eastern District,0,Noodle House,Fast Food Restaurant,Convenience Store,Cantonese Restaurant,Cha Chaan Teng,Bakery,Tram Station,Snack Place,Shopping Mall,Farmers Market


In [26]:
hk_merged.loc[hk_merged['Cluster Labels'] == 1, hk_merged.columns[[0] + list(range(3, hk_merged.shape[1]))]]

,District,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
9,Southern District,1,History Museum,Zhejiang Restaurant,Gym / Fitness Center,Grocery Store,Gourmet Shop,Gift Shop,Gastropub,Garden,Furniture / Home Store,Fried Chicken Joint


In [27]:
hk_merged.loc[hk_merged['Cluster Labels'] == 2, hk_merged.columns[[0] + list(range(3, hk_merged.shape[1]))]]

,District,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Kwai Tsing District,2,Hotel,Tunnel,Convenience Store,Thai Restaurant,Cantonese Restaurant,Chinese Restaurant,Mountain,Fast Food Restaurant,Grocery Store,Shopping Mall


In [28]:
hk_merged.loc[hk_merged['Cluster Labels'] == 3, hk_merged.columns[[0] + list(range(3, hk_merged.shape[1]))]]

,District,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Sha Tin District,3,Chinese Restaurant,Athletics & Sports,Convenience Store,Shopping Mall,Fast Food Restaurant,Pharmacy,Bus Stop,American Restaurant,Buffet,Donburi Restaurant


In [29]:
hk_merged.loc[hk_merged['Cluster Labels'] == 4, hk_merged.columns[[0] + list(range(3, hk_merged.shape[1]))]]

,District,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,Yau Tsim Mong District,4,Hotel,Cantonese Restaurant,Café,Dessert Shop,Italian Restaurant,Coffee Shop,Hong Kong Restaurant,Indian Restaurant,Park,Japanese Restaurant
7,Central and Western District,4,Coffee Shop,Japanese Restaurant,Italian Restaurant,Café,Scenic Lookout,Indian Restaurant,French Restaurant,Bar,Bakery,Trail
10,Wan Chai District,4,Japanese Restaurant,Coffee Shop,Hong Kong Restaurant,Hotel,Noodle House,Bakery,Dessert Shop,Thai Restaurant,Hotpot Restaurant,Cantonese Restaurant
